In [98]:
%run db.ipynb
from plpython_utils import plpython
schema = 'ds_training_wnv'
%read_sql SET search_path to {schema}

Query started at 06:19:58 PM EDT; Query executed in 0.00 m

# Random forest

## Create parameters table

In [99]:
#TODO: trap dummy variable
features = ['id', 'longitude','latitude', 'year', 'month', 'day', 'day_of_year',\
 'species_CULEX ERRATICUS', 'species_CULEX PIPIENS', 'species_CULEX PIPIENS/RESTUANS',\
 'species_CULEX RESTUANS', 'species_CULEX SALINARIUS', 'species_CULEX TARSALIS',\
 'species_CULEX TERRITANS', 'avg', 'preciptotal7', 'tmin7', 'tavg7', 'tmax7']
feature_formatted = ["'{}'".format(v) for v in features]  # select variables with spaces
feature_str = ','.join(feature_formatted)
n_features = len(features) - 1

In [100]:
%%read_sql
DROP TABLE IF EXISTS hyperparameters;
CREATE TABLE hyperparameters
AS
SELECT row_number() OVER () AS model_id, * 
FROM (SELECT generate_series(10,201,200) AS num_trees) a 
      CROSS JOIN (SELECT generate_series(1,{n_features}, 12) AS max_depth) b
      CROSS JOIN (SELECT generate_series(1,{n_features}, 12) AS max_features) c
-- DISTRIBUTE BY model_id;

Query started at 06:19:59 PM EDT; Query executed in 0.00 m

### Count the number of hyperparameters

In [101]:
%read_sql SELECT COUNT(*) FROM hyperparameters

Query started at 06:19:59 PM EDT; Query executed in 0.00 m

,count
0,4


# Evaluate multiple parameters

In [119]:
%load_ext autoreload
%autoreload 2
from plpython_utils import plpython

# hello world
@plpython(conn, schema, 'text')
def rf_train_tune(array1d,
                  column_names, # hello
                  dep1d,
                  id_col_name,
                  num_trees,
                  max_depth,
                  max_features):
    '''float[], text[], int[], text, int, int, int'''
    import cPickle
    
    import numpy as np
    import pandas as pd
    
    from sklearn.cross_validation import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    
    num_rows = len(array1d)/len(column_names) 
    array2d = np.reshape(array1d, (num_rows, len(column_names)))
    X = pd.DataFrame(array2d, columns = column_names)
    X = X.drop(id_col_name, axis = 1)
    y = dep1d
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
    
    cl = RandomForestClassifier(n_jobs=-1,
                                n_estimators=num_trees,
                                max_depth=max_depth,
                                max_features=max_features)
    cl.fit(X_train,y_train)
    return cPickle.dumps(cl)
    
    y_prob = cl.predict_proba(X_test)[:,1]
    return roc_auc_score(y_test, y_prob)

@plpython(conn, 'ds_training_wnv', 'setof prediction_type')
def rf_predict(array1d, column_names, model_pkl, id_col_name):
    '''float[], text[], text, text'''
    
    import cPickle
    
    import numpy as np
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    
    cl = cPickle.loads(model_pkl)
    
    num_rows = len(array1d)/len(column_names) 
    array2d = np.reshape(array1d, (num_rows, len(column_names)))
    df = pd.DataFrame(array2d, columns = column_names)
    X = df.drop(id_col_name, axis = 1)
    ids = df[id_col_name].astype(int)
    
    probs = cl.predict_proba(X)[:,1]
    
    return zip(ids, probs)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Successfully created function: ds_training_wnv.rf_train_tune("array1d" float[], "column_names"  text[], "dep1d"  int[], "id_col_name"  text, "num_trees"  int, "max_depth"  int, "max_features"  int)


DatabaseError: Execution failed on sql '
DROP FUNCTION IF EXISTS ds_training_wnv.rf_predict ("array1d" float[], "column_names"  text[], "model_pkl"  text, "id_col_name"  text);
CREATE OR REPLACE FUNCTION ds_training_wnv.rf_predict ("array1d" float[], "column_names"  text[], "model_pkl"  text, "id_col_name"  text)
RETURNS setof prediction_type
AS $$

    '''float[], text[], text, text'''
    
    import cPickle
    
    import numpy as np
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    
    cl = cPickle.loads(model_pkl)
    
    num_rows = len(array1d)/len(column_names) 
    array2d = np.reshape(array1d, (num_rows, len(column_names)))
    df = pd.DataFrame(array2d, columns = column_names)
    X = df.drop(id_col_name, axis = 1)
    ids = df[id_col_name].astype(int)
    
    probs = cl.predict_proba(X)[:,1]
    
    return zip(ids, probs)

$$ LANGUAGE plpythonu;
        ': type "prediction_type" does not exi

In [103]:
%%read_sql
SELECT version()

Query started at 06:19:59 PM EDT; Query executed in 0.00 m

,version
0,"PostgreSQL 9.6.1 on x86_64-apple-darwin14.5.0,..."


In [120]:
conn.commit()

In [121]:
%%read_sql
-- return type for modeling
DROP TYPE IF EXISTS wnv_training.prediction_type CASCADE;
CREATE TYPE prediction_type AS (
    id int,
    score float8
);

Query started at 06:21:24 PM EDT; Query executed in 0.00 mQuery started at 06:21:24 PM EDT

DatabaseError: Execution failed on sql 'CREATE TYPE prediction_type AS (
    id int,
    score float8
);': type "prediction_type" already exists


In [105]:
input_table = 'training_array'
hyperparam_table = 'hyperparameters'
output_table = 'hyperparameter_tuning_models'

In [106]:
%%read_sql
DROP TABLE IF EXISTS hyperparameter_tuning_models;
CREATE TABLE hyperparameter_tuning_models
AS
SELECT b.model_id, 
       rf_train_tune(features::float[],
                     array[{feature_str}]::text[],
                     wnv_present::int[], 'id'::text,
                     b.num_trees,
                     b.max_depth,
                     b.max_features) AS model_obj
FROM training_array a, hyperparameters b;

Query started at 06:19:59 PM EDT; Query executed in 0.02 m

In [107]:
%%read_sql df
SELECT *
FROM wnv_features
LIMIT 10

Query started at 06:20:00 PM EDT; Query executed in 0.00 m

,id,date_ts,year,month,day,day_of_year,trap,latitude,longitude,species,...,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,species_CULEX SALINARIUS,species_CULEX TARSALIS,species_CULEX TERRITANS,avg,preciptotal7,tmin7,tavg7,tmax7
0,1,2007-08-22,2007,8,22,234,T102,41.750498,-87.605294,CULEX RESTUANS,...,0,1,0,0,0,6.87500,0.430000,66.0000,72.4375,78.3750
1,3,2013-09-12,2013,9,12,255,T043,41.857402,-87.694991,CULEX PIPIENS/RESTUANS,...,1,0,0,0,0,9.21875,0.004375,67.8125,77.1875,86.0000
2,5,2007-08-01,2007,8,1,213,T135,41.662014,-87.724608,CULEX SALINARIUS,...,0,0,1,0,0,6.81250,0.080625,67.4375,76.7500,85.5625
3,7,2007-08-01,2007,8,1,213,T086,41.688324,-87.676709,CULEX PIPIENS/RESTUANS,...,1,0,0,0,0,6.81250,0.080625,67.4375,76.7500,85.5625
4,9,2013-06-07,2013,6,7,158,T151,41.916265,-87.800515,CULEX RESTUANS,...,0,1,0,0,0,8.73125,0.110625,53.3125,62.1250,70.4375
5,11,2009-09-14,2009,9,14,257,T225,41.743402,-87.731435,CULEX PIPIENS,...,0,0,0,0,0,5.50000,0.000625,61.3750,70.5625,79.2500
6,13,2011-08-19,2011,8,19,231,T069,41.768388,-87.678649,CULEX PIPIENS/RESTUANS,...,1,0,0,0,0,6.64375,0.105625,63.3125,73.1875,82.3750
7,15,2011-07-15,2011,7,15,196,T079,41.766202,-87.562889,CULEX PIPIENS/RESTUANS,...,1,0,0,0,0,7.95000,0.033750,67.4375,77.0000,85.8125
8,17,2007-08-24,2007,8,24,236,T135,41.662014,-87.724608,CULEX PIPIENS/RESTUANS,...,1,0,0,0,0,7.55625,0.660625,66.1250,73.1875,79.7500
9,19,2011-09-01,2011,9,1,244,T013,41.923738,-87.785288,CULEX PIPIENS/RESTUANS,...,1,0,0,0,0,6.58125,0.006875,63.8750,73.4375,82.5625


In [108]:
df.dtypes

id                                  int64
date_ts                            object
year                                int64
month                               int64
day                                 int64
day_of_year                         int64
trap                               object
latitude                          float64
longitude                         float64
species                            object
total_num_mosquitos                 int64
wnv_present                         int64
species_CULEX ERRATICUS             int64
species_CULEX PIPIENS               int64
species_CULEX PIPIENS/RESTUANS      int64
species_CULEX RESTUANS              int64
species_CULEX SALINARIUS            int64
species_CULEX TARSALIS              int64
species_CULEX TERRITANS             int64
avg                               float64
preciptotal7                      float64
tmin7                             float64
tavg7                             float64
tmax7                             

In [109]:
%%read_sql
DROP TABLE IF EXISTS hyperparameter_scores;
CREATE TABLE hyperparameter_scores
AS
SELECT a.*
       --b.wnv_present
FROM
    (SELECT (rf_predict(features::float[], array{features}::text[], model_obj::text, 'id'::text)).*
    FROM testing_array 
    CROSS JOIN hyperparameter_tuning_models) a
/* JOIN wnv_features b
USING (id) */

Query started at 06:20:00 PM EDT; Query executed in 0.19 m

In [110]:
%%read_sql df
SELECT *
FROM hyperparameter_scores
LIMIT 10

Query started at 06:20:12 PM EDT; Query executed in 0.00 m

,id,score
0,63,0.025992
1,139,0.049230
2,189,0.055576
3,262,0.055576
4,68,0.025992
5,328,0.025992
6,126,0.040263
7,162,0.046609
8,432,0.055576
9,228,0.025992


In [111]:
df.dtypes

id        object
score    float64
dtype: object

In [112]:
%%read_sql
SELECT *
FROM hyperparameters
JOIN hyperparameter_tuning_models
USING (model_id)

Query started at 06:20:12 PM EDT; Query executed in 0.00 m

,model_id,num_trees,max_depth,max_features,model_obj
0,1,10,1,1,ccopy_reg\n_reconstructor\np1\n(csklearn.ensem...
1,2,10,1,13,ccopy_reg\n_reconstructor\np1\n(csklearn.ensem...
2,3,10,13,1,ccopy_reg\n_reconstructor\np1\n(csklearn.ensem...
3,4,10,13,13,ccopy_reg\n_reconstructor\np1\n(csklearn.ensem...


In [113]:
df

,id,score
0,63,0.025992
1,139,0.049230
2,189,0.055576
3,262,0.055576
4,68,0.025992
5,328,0.025992
6,126,0.040263
7,162,0.046609
8,432,0.055576
9,228,0.025992


In [114]:
df.features

AttributeError: 'DataFrame' object has no attribute 'features'

In [ ]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

df = %read_sql SELECT * FROM training_array
array1d = np.array(df.features.iloc[0])
column_names = features
num_rows = len(array1d)/len(column_names) 
array2d = np.reshape(array1d, (num_rows, len(column_names)))
X = pd.DataFrame(array2d, columns = column_names)
X = X.drop('id', axis = 1)
y = df.wnv_present.iloc[0]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

cl = RandomForestClassifier(n_jobs=-1,
                            n_estimators=10,
                            max_depth=11,
                            max_features=1)
cl.fit(X_train,y_train)
y_prob = cl.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_prob)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
def plot_importances(cl, column_names, n_features=10, ax=None, error_bars = True):
    df_imp = pd.DataFrame({'features': column_names,
                           'importances': cl.feature_importances_})
    errors = np.std([tree.feature_importances_ for tree in cl.estimators_], axis=0)
    df_imp_sub = df_imp.set_index('features').sort_values('importances').tail(n_features)
    if error_bars:
        df_errors = pd.DataFrame({'features': column_names,
                                  'importances': errors})
        df_err_sub = df_errors.set_index('features').ix[df_imp_sub.index]
    else:
        df_err_sub = None
    ax = df_imp_sub.plot(kind='barh', width=.7, legend=False, ax=ax, xerr=df_err_sub, ecolor='g')
    for i,t in enumerate(df_imp_sub.index.tolist()):
        t = ax.text(0.001, i-.06,t)
        t.set_bbox(dict(facecolor='white', alpha=0.4, edgecolor='grey'))
    ax.yaxis.set_major_formatter(plt.NullFormatter())
    ax.set_title('Feature Importances')
    ax.set_xlim(0.0)
    ax.set_xlabel('importance')
    return df_imp_sub

def plot_roc(y_test, y_score, ax = None):
    if not ax:
        fig = plt.figure(figsize=(8,8))
        ax = fig.add_subplot(111)
    auc = roc_auc_score(y_test, y_score)
    fpr,tpr,thresh = roc_curve(y_test, y_score)
    ax.plot([0, 1], [0, 1], linestyle='dashed', color='grey')
    ax.plot(fpr, tpr)
    ax.set_ylabel('True Positive Rate')
    ax.set_xlabel('False Positive Rate')
    ax.set_xlim(-0.01, 1)
    ax.set_ylim(0, 1.02)
    plt.title('ROC')
    plt.text(0.85, 0.03, 'AUC: {:1.3f}'.format(auc))
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

fig, (ax,ax1) = plt.subplots(1,2, figsize = (18,8))
pred = cl.predict_proba(X_test)[:,1]
plot_roc(y_test,pred, ax=ax)
df_imp = plot_importances(cl,X.columns, error_bars=True, ax = ax1, n_features=10)

# Tune Logistic Regression

# MADlib